In [ ]:
from large_gcs.graph_generators.contact_graph_generator import (
    ContactGraphGeneratorParams,
    ContactGraphGenerator,
)
from large_gcs.graph.contact_graph import ContactGraph
from large_gcs.algorithms.gcs_astar_convex_restriction import GcsAstarConvexRestriction
from large_gcs.graph.contact_cost_constraint_factory import (
    contact_shortcut_edge_cost_factory_over_obj_weighted,
)

import numpy as np

np.set_printoptions(formatter={"float": lambda x: "{0:0.3f}".format(x)})

In [ ]:
cg = ContactGraph.load_from_file(ContactGraphGeneratorParams.graph_file_path_from_name("cg_trichal2"))

In [ ]:
# print(np.array(contact_graph.vertex_names))

In [ ]:
# vertex_inclusion = [
#     "source",
#     "('NC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f1-rob0_f1')",
#     "('IC|obs0_f0-obj0_v3', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')",
#     "('IC|obs0_v0-obj0_f2', 'NC|obs0_f0-rob0_v1', 'IC|obj0_f1-rob0_f1')",
#     "('IC|obs0_v0-obj0_f2', 'NC|obs0_f0-rob0_v1', 'NC|obj0_f0-rob0_v1')",
#     "('IC|obs0_f2-obj0_f1', 'NC|obs0_f2-rob0_v0', 'IC|obj0_f0-rob0_v1')",
#     "('IC|obs0_f2-obj0_f1', 'NC|obs0_f2-rob0_v0', 'NC|obj0_f0-rob0_v1')",
#     "('IC|obs0_f2-obj0_f1', 'NC|obs0_f2-rob0_v0', 'NC|obj0_f3-rob0_v0')",
#     "target",
# ]
# for i in range(len(vertex_inclusion) - 1):
#     has_edge_to_next_vertex = any(
#         edge.v == vertex_inclusion[i + 1]
#         for edge in contact_graph.outgoing_edges(vertex_inclusion[i])
#     )
#     print(
#         f"{vertex_inclusion[i]} -> {vertex_inclusion[i+1]}: {has_edge_to_next_vertex}"
#     )

In [ ]:
# cg.graphviz()

In [ ]:
sol = cg.solve(use_convex_relaxation=True)
print(sol.vertex_path)
print(sol.cost)
print(sol.time)

In [ ]:
contact_sol = cg.contact_spp_sol

In [ ]:
from IPython.display import HTML

anim = cg.animate_solution()
# save as mp4
# anim.save("animation.mp4")
HTML(anim.to_html5_video())